# Building a RAG Application from Scratch

All these vendors are trying to overcomplicate retrieval augmented generation (RAG). They're trying to inject different tools all over the place and make it more complicated than it needs to be. Let's change that.

This tutorial is going to teach you how to build RAG applications from scratch. No fluff, no jargon, no libraries, just a simple step by step RAG application.

Let's get started!

## The High Level Components of a RAG System

1. a collection of documents (formally called a corpus)
2. An input from the user
3. a similarity measure between the collection of documents and the user input

Yes, it's that simple. You don't need a vector store, you don't even *need* an LLM.

## The ordered steps of a RAG system

We'll perform the following steps in sequence.

1. Receive a user input
2. Perform our similarity measure.
3. Post-process the user input and the fetched document(s)


## Working through an example - the simplest RAG system

Let's walk through all of this with code examples.

### Getting a collection of documents

Below you can see that we've got a simple corpus of 'documents' (please be generous 😉).

In [67]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

### Defining and performing the similarity measure

Now we need a way of measuring the similarity between the **user input** we're going to receive and the **collection** of documents that we organized. Arguably the simplest similarity measure is [jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index). I've written about that in the past (see [this post](https://billchambers.me/posts/tf-idf-explained-in-python) but the short answer is that the **jaccard similarity** is the intersection divided by the union of the "sets" of words.

This allows us to compare our user input with the source documents.

#### Side note: preprocessing

A challenge is that if we have a plain string like `"Take a leisurely walk in the park and enjoy the fresh air.",`, we're going to have to pre-process that into a set, so that we can perform these comparisons. We're going to do this in the simplest way possible, lower case and split by `" "`.

In [73]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

Now we need to define a function that takes in the exact query and our corpus and selects the 'best' document to return to the user.

In [74]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

Now we can run it, we'll start with a simple prompt.

In [75]:
user_prompt = "What is a leisure activity that you like?"

And a simple user input...

In [76]:
user_input = "I like to hike"

Now we can return our response.

In [77]:
return_response(user_input, corpus_of_documents)

'Go for a hike and admire the natural scenery.'

Congrats, you've built a basic RAG application.


#### I got 99 problems and bad similarity is one

Now we've opted for a simple similarity measure for learning. But this is going to be problematic because it's so simple. It has no notion of **semantics**. It's just looks at what words are in both documents. That means that if we provide a negative example, we're going to get the same "result" because that's the closest document.

In [80]:
user_input = "I don't like to hike"

In [81]:
return_response(user_input, corpus_of_documents)

'Go for a hike and admire the natural scenery.'

This is a topic that's going to come up a lot with "RAG", but for now, rest assured that we'll address this problem later.

At this point, we've done zero post processing of the "document" that we're responding to. So we've really only done the "retrieval" part of "Retrieval-Augmented Generation". Let's get to augmented generation by adding a large language model (LLM).

## Adding in a LLM

To do this, we're going to use [ollama](https://ollama.ai/) to get up and running with an open source LLM on our local machine. We could just as easily use OpenAI's gpt-4 or Anthropic's Claude but for now, we'll start with the open source llama2 from [Meta AI](https://ai.meta.com/llama/).

- [ollama installation instructions are here](https://ollama.ai/)

This post is going to assume some basic knowledge of large language models, so let's get right to querying this model.

In [83]:
import requests
import json

First we're going to define the inputs. To work with this model, we're going to take 

1. user input,
2. fetch the most similar document (as measured by our similarity measure),
3. pass that into a prompt to the language model,
4. *then* return the result to the user

That introduces a new term, the **prompt**. In short, it's the instructions that you provide to the LLM.

When you run this code, you'll see the streaming result. Streaming is important for user experience.

In [95]:
user_input = "I like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
full_response = []

# https://github.com/jmorganca/ollama/blob/main/docs/api.md

prompt = """
You are a helpful bot that makes recommendations for activities. You are helpful.

This is the recommended activity: {relevant_document}

The user input is: {user_input}

Compile a recommendation to the user based on the recommended activity and the user input.
"""

Now that we've defined that, let's make the API call to ollama (and llama2).

an important step is to make sure that ollama's running already on your local machine by running `ollama serve` (crazy, I know).

In [98]:
url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            # print(decoded_line['response']) # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 Oh, I see! Thank you for letting me know! Based on your input, I would recommend a different activity Thank you for considering my suggestions! I understand that hiking may not be everyone's cup of tea, so here are some alternative activities that you might enjoy:

1. Visit a nearby park or nature reserve: Many cities have beautiful parks or nature reserves that offer a peaceful escape from the hustle and bustle of daily life. You can take a leisurely stroll, have a picnic, or simply sit and enjoy the surroundings.
2. Go for a bike ride: If you're not interested in hiking, why not try going for a bike ride? Many cities have scenic bike routes that offer a fun and active way to explore the local area.
3. Take a yoga or meditation class: Yoga and meditation can help you relax and unwind, and many studios offer classes specifically designed for beginners. This can be a great way to improve your mental and physical well-being without having to go for a hike.
4. Visit a local museum or art

This gives us a complete RAG Application, from scratch, no providers, no services. You know all of the components in a Retrieval-Augmented Generation application.

The LLM (if you're lucky) will handle the user input that goes against the recommended document. We can see that below.

In [100]:
user_input = "I don't like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
# https://github.com/jmorganca/ollama/blob/main/docs/api.md
full_response = []

prompt = """
You are a helpful bot that makes recommendations for activities. You are helpful.

This is the recommended activity: {relevant_document}

The user input is: {user_input}

Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            # print(decoded_line['response'])  # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

KeyError: 'response'

## Areas for improvement

If we go back to our diagream of the RAG application and think about what we've just built, we'll see various opportunities for improvement. These opportunities are where tools like vector stores, embeddings, and prompt 'engineering' gets involved.

Here are 10 potential areas for improvement:

1. **The number of documents** 👉 more documents might mean more recommendations.
2. **The depth/size of documents** 👉 higher quality content and longer documents with more information might be better.
3. **The number of documents we give to the LLM** 👉 Right now, we're only giving the LLM one document. We could feed in several as 'context' and allow the model to provide a more personalized recommendation based on the user input.
4. **The parts of documents that we give to the LLM** 👉 If we have bigger or more thorough documents, we might just want to add in parts of those documents, parts of various documents, or some variation there of. In the lexicon, this is called chunking.
5. **Our document storage tool** 👉 We might store our documents in a different way or different database. In particular, if we have a lot of them, we might explore storing them in a data lake or a vector store.
6. **The pre-processing of the documents & user input** 👉 We might perform some extra preprocessing or augmentation of the user input before we pass it into the similarity measure. For instance, we might use an embedding to convert that input to a vector.
7. **The similarity measure** 👉 We can change the similarity measure to fetch better or more relevant documents.
8. **The LLM/Model we use** 👉 We can change the final model that we use. We're using llama2 above, but we could just as easily use an Anthropic or Claude Model.
9. **The prompt that we use** 👉 We could use a different prompt into the LLM/Model and tune it according to the output we want to get the output we want.
10. **If you're worried about harmful or toxic output** 👉 We could implement a "circuit breaker" of sorts that runs the user input to see if there's toxic, harmful, or dangerous discussions. For instance, in a healthcare context you could see if the information contained suicidal or harm-to-others type languages and respond accordingly - outside of the typical flow.


Now improvements don't stop here. They're quite limitless and that's what we'll get into in the future. Until then, [let me know if you have any questions on twitter](https://twitter.com/bllchmbrs) and happy RAGING :).